In [ ]:
#show the file contents of the dfs
hdfs dfs -ls
#use 'hdfs dfs copyFromLocal' to transfer files from local to dfs

In [1]:
#!/usr/local/anaconda/bin/python
import sys
import re

#get lines from the input
for line in sys.stdin:
    #strip/trim any leading or trailing spaces
    line = line.strip()
    #split the line into a words array/list
    words = line.split()
    #print out each word in the words list to output
    for word in words:
        print(f"{word}\t{1}")
        
#1st mapper code

In [2]:
#!/usr/local/anaconda/bin/python
import sys

prev_key = None
prev_count = 0
word = None
curr_count = 0
#get a line from input
for line in sys.stdin:
    #strip leading/trailing spaces
    line = line.strip()
    #split the line with tab as the separator
    #with atmost one split
    word, curr_count = line.split('\t',1)
    
    #try casting the curr_count from string to int
    try:
        curr_count = int (curr_count)
    
    #if the casting doesn't work, just skip the word 
    #and go to next iteration of the loop
    except ValueError:
        continue

    #if previous word is same as current word, just increment existing count
    #with new count
    if prev_key == word:
        prev_count += curr_count
    #if the previous word is not the same as current
    #write the prev word and its count to output
    else:
        #if a previous key already exists
        #(mostly to check if prev_key had already been assigned. If its not assigned, it evaluates to false)
        if prev_key:
            print(f"{prev_key}\t{prev_count}")
        #make the current word and count assign to prev
        prev_key = word
        prev_count = curr_count

#print the last word
print(f"{prev_key}\t{prev_count}")
        
#first reducer code

In [3]:
#command to run
hadoop jar {location of streaming jar} -files mapper.py,reducer.py -mapper mapper.py -reducer reducer.py 
-input {location of the input in the hdfs directory} -output {location of where the output should be placed}

In [4]:
#!/usr/local/anaconda/bin/python

import sys

for line in sys.stdin:
    line = line.strip()
    word, count = line.split('\t', 1)
    #make count as the key
    print(f"{count}\t{word}")
    
#second mapper that takes input from the first mapreduce job

In [5]:
#!/usr/local/anaconda/bin/python
"""reducer.py"""

import sys

prev_key = None
wlist = None
curr_key = None


# input comes from STDIN
for line in sys.stdin:
    # remove leading and trailing whitespace
    line = line.strip()

    # parse the input we got from mapper.py
    curr_key, word = line.split('\t', 1)
    
    try:
        curr_key = int (curr_key)
        
    except ValueError:
        continue
        
    if prev_key == curr_key:
        #add to the word list when both words have the same count
        wlist.append(word)
        
    else:
        #output the prev key with its associated list
        print(f"{prev_key}\t{wlist}")
        #create new list
        wlist = []
        #assign current key (count) as prev key
        prev_key = curr_key
        #append the current word to word list
        wlist.append(word)
     
print(f"{prev_key}\t{wlist}")

#second reducer that takes the second mapper output

None	None


In [ ]:
#jardir = location of the hadoop-streaming.jar
#hdinp = location of input in the dfs
hadoop jar $jardir -D mapred.output.key.comparator.class=org.apache.hadoop.mapred.lib.KeyFieldBasedComparator \
-D mapred.text.key.comparator.options=-k1,1nr -files mapper2.py,reducer2.py -mapper mapper2.py \
-reducer reducer2.py -input $hdinp -output rating1